In [32]:
from sklearn.neighbors import NearestNeighbors

class Nbdos:
	
    def __init__(self):
        self.hck = {}

    def list_neighbor(self, data, sfs, xi, k):
        neigh = NearestNeighbors(n_neighbors=k)
        neigh.fit(data.iloc[sfs, :data.shape[1]], data.iloc[sfs, -1])
        return neigh.kneighbors([xi], return_distance=False).tolist()[0]

    def nbdos(self, data, sc, k, k2_neighbors, rTh, nTh):
        sfs = []
        cl = [0] * sc.shape[0]
        #print("aqui foi")
        return cl

        # for indeces, xi in sc.iterrows():
        #     tem = [a for a in self.list_neighbor(data, k2_neighbors, xi, k) if data.iloc[a, -1] == xi.iloc[-1]]
        #     #print(indeces)
        #     #print(len(tem))
        #     if round(len(tem)/k) >= rTh:
        #         print("Nem se quer aqui")
        #         sfs.append(indeces)
        #         self.hck[indeces] = tem
        
        # for xi in sfs:
        #     tem = self.list_neighbor(sfs, xi, cl, k)
        #     print("passa aqui")
        #     if xi in self.hck.keys():
        #         self.hck[xi].append(tem)
        #     else:
        #         self.hck[xi] = tem

        # #print(list(sfs))
        # curld = 0
        # for xi in sfs:
        #     if cl[xi] == 0:
        #         curld += 1
        #         self.expand_cluster(sfs, xi, cl, curld) #voltar a partir daqui

        # for i in range(curld):
        #     ci = [x for x in range(len(cl)) if cl[x] == i]
        #     #ci = [xi for j, xi in sc.iterrows() if cl == i]

        #     if len(ci) < nTh:
        #         for j in ci:
        #             cl[j] = 0
        
        # return cl 
    
    def expand_cluster(self, sfsc, xi, cl, curld):
        sfC = [xi]
        cl[xi] = curld

        # Check if list is empty
        while sfC:
            xj = sfC.__getitem__(-1)
            for xl in self.hck[xj]:
                if xl.cl == 0:
                    xl.cl = curld
                    if xl in sfsc:
                        sfC.append(xi)

            sfC.remove(xj)
        return cl


In [33]:
import operator
import math
import numpy
from functools import reduce
import pandas as pd
import numpy as np
class Instance:

    def __init__(self, label):
        # tuple key where key is the neighbor and the value is the selection weight
        self.neighbors = {}
        self.nk1 = {}
        self.Fs_Fd = {}
        self.cl = 0
        self.label = label

    def set_fs_fd(self, fs):
        self.Fs_Fd = fs

    def get_fs(self):
        return list(self.Fs_Fd.keys())

    def get_fd(self):
        return list(self.Fs_Fd.values())

    def get_nk1(self):
        return self.nk1.keys()

    def add_neighbor(self, neighbor):
        self.neighbors[neighbor] = 0

    def get_list_neighbor(self):
        return list(self.neighbors.keys())

    def get_neighbor_weight(self, neighbor):
        return self.neighbors[neighbor]

    def get_neighbor_high_weight(self):
        return max(self.neighbors, key=self.neighbors.get)

    def set_selection_weight(self, key, w1):
        self.neighbors[key] = (1 + w1)/math.e

    @staticmethod
    def ss(xi, xj, fs_fd):
        """Step 5.3 article
            :return list of neighbors Fs which the distance is no longer than
            the distance between this instance and the parameter
        """
        threshold = numpy.linalg.norm(xi - xj)
        return list({k: v for k, v in fs_fd.items() if numpy.linalg.norm(xi-v) > threshold}.keys())

    @staticmethod
    def dpn(pri, ari, ss, data):
        """
        Used to find the instances set, NPN , which belong to the PN neighborhood xi.P N(xj)
        Step 5.3 article + supplementary material
        :param pri: primary reference instance (xi)
        :param ari: assistant reference instance (xj)
        :param ss: returned set from function ss
        :return: the Npn: the set of instances in a radius distance from xi and xj
        """

        npn = [pri, ari]
        midaux = tuple(map(operator.add, pri, ari))
        mid = tuple(map(lambda x: operator.truediv(x, 2), midaux))
        dist_pri_ari = reduce(operator.add, pri+ari)

        for xl in ss:
            x = data.iloc[xl]
            dist_mid_xl = numpy.linalg.norm(x - mid) 
            if dist_mid_xl <= dist_pri_ari:
                npn.append(xl)

        return npn
    @staticmethod
    def class_entropy(gama_mi, class_set):
        gama_m = class_set.shape[0]
        ret = 0

        for index, c in class_set.iterrows():
            ret += pd.Series.div((pd.Series.div(c, gama_mi) * pd.Series.div(c, gama_mi).apply(np.log10)), np.log10(gama_m))

        return ret
    
    @staticmethod
    def selection_weight_formula(npn, w1, w2, r1, r2, mc, ma, mi, minorities):
        """formula (2) article
            :param mc class c instances
            :param ma majority class instance
            :param mi minority class instance
            :param minorities minorities classes"""

        gama_ma = ma.shape[0]
        gama_mi = mi.shape[0]
        gama_c = mc.shape[0]

        # Minority classes entropy
        e_mi = Instance.class_entropy(gama_mi, minorities)

        # Majority class entropy
        e_ma = Instance.class_entropy(gama_mi, minorities)

        overgeneralization_factor_aux = r1 * (gama_mi/ gama_c) + r2 * e_mi + w2 * (r1* gama_ma / gama_c + 2 * e_ma)
        overgeneralization_factor = overgeneralization_factor_aux.apply(lambda x: x**2)
        difficulty_factor = math.exp(-1 * (gama_c / gama_c + gama_ma + gama_mi))

        return 1 / (overgeneralization_factor + w1 * difficulty_factor)


In [34]:
import pandas as pd
import math
import random as rd
from sklearn.neighbors import NearestNeighbors
import numpy
import pandas as pd
from instance import Instance

class SMOM:
    def __init__(self):
        self.outstanding = []
        self.trapped = []

    @staticmethod
    def kneigbor(data, xi, k, return_distance):
        knn = NearestNeighbors(n_neighbors=k)
        knn.fit(data.iloc[:, :data.shape[1]], data.iloc[:, -1])
        return knn.kneighbors([xi], return_distance=return_distance)
        


    @staticmethod
    def split_classes(data, minority_class):
        c = data[data.iloc[:, -1] == minority_class]
        not_c = data[data.iloc[:, -1] != minority_class]
        return c, not_c

    @staticmethod
    def nearestK3Instances(xi, sc, k1, k2):
        k3 = max([k1,k2])    

        knn3 = NearestNeighbors(n_neighbors=k3)
        knn3.fit(sc.iloc[:, :sc.shape[1]], sc.iloc[:, -1])
        dist, k3neighbors = knn3.kneighbors([xi], return_distance=True)
        
        k3neighbors = k3neighbors.tolist()[0]
        dist = dist.tolist()[0]

        knn1 = NearestNeighbors(n_neighbors=k1)
        knn1.fit(sc.iloc[:, :sc.shape[1]], sc.iloc[:, -1])
        k1neighbors = knn1.kneighbors([xi], return_distance=False).tolist()[0]

        # k3neighbors = knn.getNeighbors(sc, xi, k3)
        # k1neighbors = knn.getNeighbors(sc, xi, k1)        
        k1th = k1neighbors[0]
        distance = numpy.linalg.norm(xi-k1th)
    
        return k3neighbors, dist, k1th, distance, k3

    @staticmethod
    def selection_weigth(data, outstandings, trapped_instances, k3, w1, w2, r1, r2, xi_fs_fd, minor):
        """ Return a dict of weight for each instance
            @:param trapped_instance list of type instance
        """
        knn3 = NearestNeighbors(n_neighbors=k3)

        sw = {}

        for xi in trapped_instances:
            index = data[data == xi].dropna(axis=0).index[0] 

            for xj in xi_fs_fd[index].keys():
                xi_sw = {}

                vis_xj = xi_fs_fd.get(xj, {})
                # print(xi)
                if (xj in outstandings) and (xi in xi_fs_fd[xj].keys()):
                    xi_sw[xj] = 1 + w1/math.e              
                elif (xj in xi_fs_fd.keys()) and (index in xi_fs_fd[xj].keys()) and (xj in sw.keys()) and (index in sw[xj].keys()):
                    print("foi")
                    sw[index][xj] = sw[xj][index]
                else:    
                    v_xj = data.iloc[xj]
                    smaller_distances = Instance.ss(xi, v_xj, xi_fs_fd[index]) 
                    npn = Instance.dpn(xi, v_xj, smaller_distances, data)
                    
                    vals = data[data.iloc[:, -1] == minor]
                    ma_class, mi_class, minorities_class_set = SMOM.get_classes(data)
                    ma = SMOM.get_class_set(data, ma_class)
                    mi = SMOM.get_class_set(data, minor)

                    minorities = data[(data.iloc[:, -1] == 1) | (data.iloc[:, -1] == 25) | (data.iloc[:, -1] == 2) | (data.iloc[:, -1] == 26) | (data.iloc[:, -1] == 29)]
                    # minorities_class_set = {}
                    ret = Instance.selection_weight_formula(npn, w1, w2, r1, r2, vals, ma, mi, minorities)
                    #print(ret)
                    xi_sw[xj] = ret
               
            sw[index] = xi_sw
            print("foi aqui")
        
        # return weightDict


    @staticmethod
    def filterOutstanding(sc, cl, minor):
        outstanding = []
        trapped = []
        index_trapped = []
        
        for instanceIndex in range(sc[sc.iloc[:, -1] == minor].shape[0]):
            #print(instanceIndex)
            instance = sc.iloc[instanceIndex]
            if cl[instanceIndex] != 0:
                outstanding.append(instance)
            else:
                trapped.append(instance)
                index_trapped.appens(instanceIndex)
                
        return (outstanding, trapped, index_trapped)

    @staticmethod
    def get_classes(data):
        # Get majority class
        ma = data.iloc[:, -1].value_counts().idxmax()

        # Get minority class
        mi = data.iloc[:, -1].value_counts().idxmin()

        # Get minority classes set
        mi_aux = data.iloc[:, -1].value_counts() == data.iloc[:, -1].value_counts().min()
        mi_set = mi_aux[mi_aux == True]
        return ma, mi, mi_set
    
    @staticmethod
    def get_class_set(data, class_name):
        # print(">>> " + str(data.iloc[:, -1].size) )
        # print(class_name)
        return data[data.iloc[:, -1] == class_name]

    def generate_synthetic_instances(self, sc, g):
        xj = 0
        si = []
        for times in range(g):
            for i in sc:
                if i in self.trapped:
                    xj = i.get_neighbor_high_weight()
                else:
                    xj = rd.choice(i.get_nk1())

                diff = (xj - i)
                gama = [rd.randrange(0, 2) for i in range(len(xj))]
                new_instance = i + diff.dot(gama)

                si.append(new_instance)

        return si

    @staticmethod
    def get_g_for_each_xi(xi, zeta, sc):
        g, remainder = SMOM.get_floor_remainder(zeta, len(sc))

        if xi == sc[-1]: ##
            return g+remainder
        else:
            return g

    @staticmethod
    def get_floor_remainder(zeta, sc_size):
        if (zeta%sc_size) == 0:
            return zeta/sc_size, 0
        else:
            g_for_each_xi = int(math.floor(zeta/sc_size))
            remainder = zeta%sc_size
            return g_for_each_xi, remainder

    @staticmethod
    def nearEnemy(xi, others, k3, distance, neighbors, neighbors_distance):
        knn3 = NearestNeighbors(n_neighbors=k3)
        knn3.fit(others.iloc[:, :others.shape[1]], others.iloc[:, -1])
        dist, k3neighbors = knn3.kneighbors([xi], return_distance=True)
        
        k3neighbors = k3neighbors.tolist()[0]
        dist = dist.tolist()[0]

        fs = []
        fd = []
        for i in range(len(dist)):
            if dist[i] < distance:
                fs.append(k3neighbors[i])
                fd.append(dist[i])

        fs = fs + neighbors
        fd = fd + neighbors_distance
        
        return k3neighbors, fs, fd        
        
    @staticmethod
    def k2_neighbors(data, k3_neighbor, k3_enemy, xi, k2, k3):
        frames = numpy.concatenate([k3_neighbor, k3_enemy])
        #print(frames)
        knn = NearestNeighbors(n_neighbors=k2)
        knn.fit(data.iloc[frames, :len(frames)], data.iloc[frames, -1])
        return knn.kneighbors([xi], return_distance=False).tolist()[0] 

    @staticmethod
    def probability_distribution(xi, index_xi, k1neighbors, w1, fs_fd, data): # the return is a list of probabilities with the ormat [xj, xipj], xj is the instance from a different class and xipj it probability
        xipj = [] 
        flag = True
        weightlist = []

        for xj in k1neighbors: #k1neighbors should follows the same format of the return of this method: i.e: [[[0.00078,0.9475,0.084,0.00009,"g"], 0.002568],[[0.045,0.0645,0.084,0.00009,"t"], 0.00568]]  while xi is a simple instance without the distance
            weightlist.append(fs_fs.get(xi))
            smaller_distances= Instance.ss(xi, xj, fs_fd)
            npn = Instance.dpn(xi, xj, smaller_distances, data)
                
            for element in npn:
                print(element)
                if element[-2][-1] == xi[-1]:
                    flag = False
                    break
                
            if(xj[-2][-1] != xi[-1]):
                    xipj.append([xj,xi.get_neighbor_weight(xj)/(math.sum(weightlist))])

        if flag:
            weight = 1 + (w1/math.e)
            k1neighbors.append([xi, weight])
            
        return xipj

In [35]:
import pandas as pd
from SMOM import SMOM
from nbdos import Nbdos
import pandas as pd
# import main2

data = pd.read_csv("data/data.csv", header=None)
data = data.iloc[:, 1:].head(500)
#min_class = data.iloc[:, -1].value_counts().min()
min_class = 16 #classe com 67 instancias
k1 = 12
k2 = 8
rTh = 5/8
nTh = 10
k = 2
w1 = 0.2
w2 = 0.5
r1 = 1/3
r2 = 0.2

#1. Dividindo dados entre classe minoriataria e outros
sc, others =  SMOM.split_classes(data, min_class) 

xi_fs_fd = {}

#2.
for index, xi in sc.iterrows():
    fs_fd = {}
# 2.1)
    k3neighbors, neighbors_distance, k1th, distance, k3 = SMOM.nearestK3Instances(xi, sc, k1, k2)
# 2.2)
    k3EnemyNeighbor, fs, fd = SMOM.nearEnemy(xi, others, k3, distance, k3neighbors, neighbors_distance)
# 2.3)
    k2_neighbors = SMOM.k2_neighbors(data, k3neighbors, k3EnemyNeighbor, xi, k2, k3)
    
    for i in range(len(fs)):
        fs_fd[fs[i]] = fd[i]
    xi_fs_fd[index] = fs_fd
#3.
cl = Nbdos().nbdos(data, sc, k2, k2_neighbors, rTh, nTh)
#4.
outstandings, trappeds, index_trapped = SMOM.filterOutstanding(sc, cl, min_class)       
#5.
dic = SMOM.selection_weigth(data, outstandings, trappeds, k3, w1, w2, r1, r2, xi_fs_fd, min_class)

#6.



#  6.1)
#  6.2)


#7.

#8.
# instance
#   8.1.
#   8.2
#   8.3.


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
for i in range(len(trappeds)):
    
    k1neighbors = SMOM.kneigbor(data, trappeds[i], k1, False)
    xipj = SMOM.probability_distribution(xi, index_trapped, k1neighbors, w1, data, xi_fs_fd.get(index_trapped))
    print(xipj)

In [ ]:
for i in range(sc[sc.iloc[:, -1] == 16].shape[0]):
    print(i)
    

In [39]:
def a():
    return  1, 2, 3

c,d,e = a()
e

3